# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1_csv_file = "../WeatherPy/random_cities_weather_data_file.csv"
random_cities_weather_df = pd.read_csv(part1_csv_file)
random_cities_weather_df = random_cities_weather_df.drop(columns=["Unnamed: 0"])


In [3]:

random_cities_weather_df.head(4)

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed
0,saint-philippe,re,-56.221530,66.735628,76.82,65,20,23.04
1,dukuhturi,id,-6.836002,108.847204,87.12,64,69,6.13
2,nuuk,gl,65.340080,-50.845383,30.09,86,89,4.70
3,new norfolk,au,-67.252573,128.168859,57.18,63,40,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
cities_humidity_df = random_cities_weather_df["Humidity"].astype(float)
cities_humidity_max_df = cities_humidity_df.max()
cities_location_df = random_cities_weather_df[["Latitude","Longititude"]]

In [6]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities_location_df, weights=cities_humidity_df,
                                 dissipating=False, max_intensity=cities_humidity_max_df , point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### New DataFrame fitting weather criteria
- Tempratuture greater than 70 and less than 80
- wind speed less than 10mph
- zero cloudiness

In [15]:
condition1 = random_cities_weather_df["cloudiness"] == 0
condition2 = random_cities_weather_df["Temprature"] >50 
condition3 = random_cities_weather_df["Temprature"] <=80
condition4 = random_cities_weather_df["wind_speed"] <=10
mask_df = random_cities_weather_df[condition1 & condition2 & condition3 & condition4 ].reset_index(drop=True)
mask_df

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed
0,cape town,za,-59.045523,-4.950902,63.30,77,0,9.17
1,port alfred,za,-89.745337,43.939570,69.35,64,0,7.07
2,petropavlovsk-kamchatskiy,ru,50.437335,159.511476,51.80,32,0,4.47
3,tessalit,ml,22.315007,3.824591,78.24,17,0,6.04
4,murgab,tm,37.277700,62.728764,57.20,44,0,6.93
5,port elizabeth,za,-86.132761,32.148474,66.20,68,0,3.36
6,eregli,tr,37.359392,34.399306,61.70,46,0,0.18
7,xining,cn,35.366935,101.351054,54.30,33,0,3.04
8,jumla,np,33.094356,84.472604,66.60,27,0,6.44
9,qena,eg,26.801000,32.653905,75.20,38,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = random_cities_weather_df[condition1 & condition2 & condition3 & condition4 ]
hotel_df = hotel_df.copy()
hotel_df["Hotel Name"] =''
hotel_df.reset_index(inplace= True,drop = True)
hotellist= []
hotel_df

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed,Hotel Name
0,cape town,za,-59.045523,-4.950902,63.30,77,0,9.17,
1,port alfred,za,-89.745337,43.939570,69.35,64,0,7.07,
2,petropavlovsk-kamchatskiy,ru,50.437335,159.511476,51.80,32,0,4.47,
3,tessalit,ml,22.315007,3.824591,78.24,17,0,6.04,
4,murgab,tm,37.277700,62.728764,57.20,44,0,6.93,
5,port elizabeth,za,-86.132761,32.148474,66.20,68,0,3.36,
6,eregli,tr,37.359392,34.399306,61.70,46,0,0.18,
7,xining,cn,35.366935,101.351054,54.30,33,0,3.04,
8,jumla,np,33.094356,84.472604,66.60,27,0,6.44,
9,qena,eg,26.801000,32.653905,75.20,38,0,2.24,


In [52]:
hotellist= []
i = 0
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Latitude"]
    lng = hotel_df.loc[i]["Longititude"]
    params = {
            "location": f"{lat},{lng}",
            "language": "en",
            "radius": 5000,
            "types": "hotel",         
            "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params)
    data = response.json()
    if data['status'] == "OK":
        #data['results'][0]['name']
        #print("OK if")
        
        hotel_df.loc[i,"Hotel Name"] = data['results'][0]['name']
        xxxx = data['results'][0]['name']
        #print(f'For {i} row value is : {xxxx}')
    else:
        hotellist.append("")
        #print(f'Appended : Blank')
        

#hotellist[2] = "Non English response"
#hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head(20)


,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed,Hotel Name
0,cape town,za,-59.045523,-4.950902,63.30,77,0,9.17,
1,port alfred,za,-89.745337,43.939570,69.35,64,0,7.07,
2,petropavlovsk-kamchatskiy,ru,50.437335,159.511476,51.80,32,0,4.47,
3,tessalit,ml,22.315007,3.824591,78.24,17,0,6.04,Abalessa
4,murgab,tm,37.277700,62.728764,57.20,44,0,6.93,
5,port elizabeth,za,-86.132761,32.148474,66.20,68,0,3.36,
6,eregli,tr,37.359392,34.399306,61.70,46,0,0.18,
7,xining,cn,35.366935,101.351054,54.30,33,0,3.04,Hainan Tibetan Autonomous Prefecture
8,jumla,np,33.094356,84.472604,66.60,27,0,6.44,Ngari Prefecture
9,qena,eg,26.801000,32.653905,75.20,38,0,2.24,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
